# Neural Networks Lesson 1B: Handwritten Digit Recognition (Enhanced)

## Building a Real-World Image Classifier - Step by Step with Detailed Explanations

**Learning Objectives:**
- **Deeply understand** how images become neural network inputs (with visualizations!)
- Master the concept of "flattening" - turning a 2D image into a 1D vector
- Visualize how 784 pixels connect to 784 input neurons
- Work with the famous MNIST dataset
- Build and train a neural network for classification
- Understand every step with concrete examples and graphs

**Duration:** ~120 minutes (with time to explore and understand)

**Why This Matters:** This lesson prepares you for the text classification assignment. The same principles apply: whether we're processing images or text, we need to convert data into vectors that neural networks can understand!

---

## Part 1: Introduction to MNIST - The "Hello World" of Computer Vision

MNIST (Modified National Institute of Standards and Technology) is a dataset of handwritten digits that has been used to train and test machine learning models since the 1990s.

### Dataset Facts:
- **70,000 images** of handwritten digits (0-9) written by real people
- **60,000 training images** (we use these to teach the network)
- **10,000 test images** (we use these to evaluate how well it learned)
- Each image is **28×28 pixels** in grayscale (black and white)
- **784 total pixels** per image (28 × 28 = 784)
- Pixel values range from **0 (black) to 255 (white)**

### Why MNIST?
- Small enough to train quickly on a laptop
- Complex enough to demonstrate real neural network concepts
- Universal benchmark - you can compare your results with thousands of other implementations
- Perfect stepping stone to understanding how modern computer vision works!

In [ ]:
# Setup: Install and import required libraries
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns

# Set random seeds for reproducibility (so you get the same results every time)
np.random.seed(42)

print("✅ Libraries imported successfully!")
print(f"NumPy version: {np.__version__}")
print("\n📚 Ready to learn about image classification!")

## Part 2: Load and Explore the MNIST Dataset

In [ ]:
# Load MNIST dataset
print("📥 Loading MNIST dataset (this may take a moment - it's downloading 70,000 images!)...\n")
mnist = fetch_openml('mnist_784', version=1, parser='auto')
X, y = mnist.data.values, mnist.target.values.astype(int)

print(f"✅ Dataset loaded!")
print(f"\n📊 Dataset Statistics:")
print(f"  • Total images: {X.shape[0]:,}")
print(f"  • Pixels per image: {X.shape[1]} (this is 28×28 = 784)")
print(f"  • Classes: {len(np.unique(y))} (digits 0-9)")
print(f"  • Data type: {X.dtype}")
print(f"  • Value range: [{X.min():.0f}, {X.max():.0f}]")
print(f"\n🔍 Shape of X: {X.shape}")
print(f"   Translation: {X.shape[0]} images, each with {X.shape[1]} pixel values")

In [ ]:
# Visualize sample images to understand what we're working with
fig, axes = plt.subplots(2, 5, figsize=(14, 6))
fig.suptitle('Sample MNIST Handwritten Digits', fontsize=18, fontweight='bold')

for i, ax in enumerate(axes.flat):
    # Reshape the flat 784-element array back into 28×28 for display
    image = X[i].reshape(28, 28)
    ax.imshow(image, cmap='gray')
    ax.set_title(f'Label: {y[i]}', fontsize=14, fontweight='bold', color='blue')
    ax.axis('off')

plt.tight_layout()
plt.show()

print("\n👀 What you're seeing:")
print("  • Real handwritten digits from different people")
print("  • Different writing styles and shapes")
print("  • Some are clear, some are messy (just like real handwriting!)")
print("  • Our neural network will learn to recognize ALL of these variations!")

## Part 3: UNDERSTANDING IMAGE FLATTENING - The Most Important Concept!

### 🎯 This is where many students get confused. Let's break it down step by step!

**The Challenge:** Neural networks need 1D vectors as input, but images are 2D grids!

**The Solution:** We "flatten" the 2D grid into a 1D vector.

### What Does "Flattening" Mean?

Imagine you have a chessboard (8×8 = 64 squares). Flattening means:
1. Take the first row and write it down: squares 1-8
2. Take the second row and write it after: squares 9-16
3. Continue for all rows...
4. Now you have a LINE of 64 squares instead of a grid!

For MNIST:
- Original: 28 rows × 28 columns = **2D grid**
- Flattened: 784 elements in a **1D line**
- **Same information, different shape!**

In [ ]:
# Let's demonstrate flattening with a SIMPLE example first
# We'll use a tiny 5×5 image to make it easy to understand

print("📐 DEMONSTRATION: Flattening a Simple 5×5 Grid\n")

# Create a simple 5×5 pattern
simple_image = np.array([
    [0, 0, 255, 0, 0],
    [0, 255, 255, 255, 0],
    [255, 255, 255, 255, 255],
    [0, 255, 255, 255, 0],
    [0, 0, 255, 0, 0]
])

print("ORIGINAL 2D IMAGE (5×5):")
print(simple_image)
print(f"Shape: {simple_image.shape}\n")

# Flatten it
flattened = simple_image.flatten()

print("FLATTENED 1D VECTOR (25 elements):")
print(flattened)
print(f"Shape: {flattened.shape}\n")

# Visualize the process
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Show 2D image
axes[0].imshow(simple_image, cmap='gray', vmin=0, vmax=255)
axes[0].set_title('BEFORE: 5×5 Grid (2D)\n25 values arranged in rows and columns', 
                  fontsize=13, fontweight='bold')
axes[0].grid(True, color='red', linewidth=2)
axes[0].set_xticks(range(5))
axes[0].set_yticks(range(5))

# Show 1D flattened version
axes[1].imshow(flattened.reshape(1, -1), cmap='gray', vmin=0, vmax=255, aspect='auto')
axes[1].set_title('AFTER: 25-element Vector (1D)\nSame 25 values in a single line', 
                  fontsize=13, fontweight='bold')
axes[1].set_xlabel('Position (0-24)', fontsize=11)
axes[1].set_yticks([])

plt.tight_layout()
plt.show()

print("\n🔍 KEY INSIGHT:")
print("  ✓ Same pixel values (same brightness)")
print("  ✓ Same information (same pattern)")
print("  ✓ Different shape (2D → 1D)")
print("  ✓ Now compatible with neural network input!")

## Part 4: Flattening a REAL MNIST Image (28×28)

Now let's apply the same concept to a real MNIST digit!

In [ ]:
# Pick a real MNIST digit and demonstrate flattening
sample_idx = 0
sample_image_2d = X[sample_idx].reshape(28, 28)  # 2D version (28×28)
sample_image_1d = X[sample_idx]                   # 1D version (784)

print(f"🔍 Examining MNIST digit '{y[sample_idx]}':\n")
print(f"ORIGINAL IMAGE:")
print(f"  • Shape: {sample_image_2d.shape} (28 rows, 28 columns)")
print(f"  • Total pixels: 28 × 28 = {28*28}")
print(f"  • This is a 2D GRID\n")

print(f"FLATTENED IMAGE:")
print(f"  • Shape: {sample_image_1d.shape} (784 elements in a line)")
print(f"  • This is a 1D VECTOR")
print(f"  • First 20 pixel values: {sample_image_1d[:20]}")
print(f"\n💡 The flattened version has the SAME pixel values, just rearranged into a line!")

In [ ]:
# Visualize HOW flattening works row-by-row
fig = plt.figure(figsize=(16, 10))

# Top: Show the 2D image
ax1 = plt.subplot(3, 1, 1)
im = ax1.imshow(sample_image_2d, cmap='gray')
ax1.set_title(f'STEP 1: Original 28×28 Image (Digit: {y[sample_idx]})', 
              fontsize=16, fontweight='bold', pad=20)
ax1.set_xlabel('Column (0-27)', fontsize=12)
ax1.set_ylabel('Row (0-27)', fontsize=12)
plt.colorbar(im, ax=ax1, label='Pixel Intensity (0=black, 255=white)')

# Middle: Show how rows stack
ax2 = plt.subplot(3, 1, 2)
# Show a few rows highlighted
ax2.imshow(sample_image_2d, cmap='gray', alpha=0.3)
ax2.axhline(y=0.5, color='red', linewidth=3, label='Row 0 (first 28 pixels)')
ax2.axhline(y=5.5, color='orange', linewidth=3, label='Row 5')
ax2.axhline(y=13.5, color='yellow', linewidth=3, label='Row 13 (middle)')
ax2.set_title('STEP 2: Each ROW becomes a segment of the flattened vector', 
              fontsize=16, fontweight='bold', pad=20)
ax2.legend(loc='upper right', fontsize=10)
ax2.set_xlabel('28 pixels per row', fontsize=12)
ax2.set_ylabel('28 rows total', fontsize=12)

# Bottom: Show the flattened vector with row markers
ax3 = plt.subplot(3, 1, 3)
ax3.imshow(sample_image_1d.reshape(1, -1), cmap='gray', aspect='auto')
ax3.set_title('STEP 3: Flattened into 784-element vector (28 rows × 28 pixels = 784)', 
              fontsize=16, fontweight='bold', pad=20)
ax3.set_xlabel('Position in vector (0-783)', fontsize=12)
ax3.set_yticks([])

# Add markers showing where each row starts
for row in [0, 5, 13, 27]:
    pos = row * 28
    ax3.axvline(x=pos, color='red', linewidth=2, alpha=0.5)
    ax3.text(pos, 0.5, f'Row {row}', rotation=90, fontsize=9, 
             verticalalignment='center', color='red', fontweight='bold')

plt.tight_layout()
plt.show()

print("\n📊 HOW FLATTENING WORKS:")
print("  Row 0 (pixels 0-27)   → Vector positions 0-27")
print("  Row 1 (pixels 0-27)   → Vector positions 28-55")
print("  Row 2 (pixels 0-27)   → Vector positions 56-83")
print("  ...")
print("  Row 27 (pixels 0-27)  → Vector positions 756-783")
print("\n🎯 RESULT: One long line with all 784 pixel values!")

In [ ]:
# Show pixel intensity profile to make the data more concrete
fig, axes = plt.subplots(2, 2, figsize=(16, 10))

# Top-left: The image
axes[0, 0].imshow(sample_image_2d, cmap='gray')
axes[0, 0].set_title(f'Digit: {y[sample_idx]}', fontsize=14, fontweight='bold')
axes[0, 0].axhline(y=14, color='red', linewidth=2, linestyle='--', label='Row 14')
axes[0, 0].axvline(x=14, color='blue', linewidth=2, linestyle='--', label='Column 14')
axes[0, 0].legend()
axes[0, 0].set_xlabel('Column', fontsize=11)
axes[0, 0].set_ylabel('Row', fontsize=11)

# Top-right: One row's pixel values
row_14 = sample_image_2d[14, :]
axes[0, 1].plot(row_14, 'r-', linewidth=2, marker='o', markersize=4)
axes[0, 1].set_xlabel('Column Position (0-27)', fontsize=11)
axes[0, 1].set_ylabel('Pixel Intensity', fontsize=11)
axes[0, 1].set_title('Pixel Values in Row 14 (28 values)', fontsize=14, fontweight='bold')
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].set_ylim(-10, 265)

# Bottom-left: One column's pixel values
col_14 = sample_image_2d[:, 14]
axes[1, 0].plot(col_14, 'b-', linewidth=2, marker='o', markersize=4)
axes[1, 0].set_xlabel('Row Position (0-27)', fontsize=11)
axes[1, 0].set_ylabel('Pixel Intensity', fontsize=11)
axes[1, 0].set_title('Pixel Values in Column 14 (28 values)', fontsize=14, fontweight='bold')
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].set_ylim(-10, 265)

# Bottom-right: First 100 pixels of flattened vector
axes[1, 1].plot(sample_image_1d[:100], 'g-', linewidth=1.5)
axes[1, 1].set_xlabel('Position in Flattened Vector', fontsize=11)
axes[1, 1].set_ylabel('Pixel Intensity', fontsize=11)
axes[1, 1].set_title('First 100 Values of Flattened Vector', fontsize=14, fontweight='bold')
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].set_ylim(-10, 265)

plt.tight_layout()
plt.show()

print("\n🔬 UNDERSTANDING THE DATA:")
print(f"  • Row 14 has {len(row_14)} pixel values")
print(f"  • Column 14 has {len(col_14)} pixel values")
print(f"  • Flattened vector has {len(sample_image_1d)} total values (28 rows × 28 columns)")
print(f"  • Each spike in the graphs represents a bright pixel (part of the digit)")

## Part 5: FROM FLATTENED VECTOR TO NEURAL NETWORK INPUT

### 🧠 This is the connection students often miss!

**Question:** We have a 784-element vector. Now what?

**Answer:** Each element of the vector connects to ONE neuron in the input layer!

### The Architecture:
```
Image (28×28)  →  Flatten  →  784-element vector  →  784 Input Neurons
   ┌─────┐         ────>       [p₀, p₁, ..., p₇₈₃]        ●  ●  ●
   │ ■ □ │                                               ●  ●  ●
   │ □ ■ │                                              ●  ●  ●
   └─────┘                                            (784 neurons)
```

**Each pixel value goes to exactly ONE input neuron!**
- Pixel 0 → Neuron 0
- Pixel 1 → Neuron 1
- ...
- Pixel 783 → Neuron 783

In [ ]:
# Visualize the neural network architecture
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Left: The image
axes[0].imshow(sample_image_2d, cmap='gray')
axes[0].set_title('1. Original Image\n(28×28 = 784 pixels)', 
                  fontsize=13, fontweight='bold')
axes[0].axis('off')

# Middle: The flattened vector
axes[1].imshow(sample_image_1d.reshape(1, -1), cmap='gray', aspect='auto')
axes[1].set_title('2. Flattened Vector\n(784 elements in a line)', 
                  fontsize=13, fontweight='bold')
axes[1].set_xlabel('Index: 0 → 783', fontsize=11)
axes[1].set_yticks([])

# Right: Representation of input layer
# We'll show a subset of neurons to make it visible
axes[2].text(0.5, 0.9, 'Input Layer\n(784 neurons)', 
            ha='center', va='top', fontsize=14, fontweight='bold',
            transform=axes[2].transAxes)
axes[2].text(0.5, 0.75, '●', ha='center', va='center', fontsize=40, 
            transform=axes[2].transAxes, color='blue')
axes[2].text(0.3, 0.65, '●', ha='center', va='center', fontsize=40, 
            transform=axes[2].transAxes, color='blue')
axes[2].text(0.7, 0.65, '●', ha='center', va='center', fontsize=40, 
            transform=axes[2].transAxes, color='blue')
axes[2].text(0.2, 0.55, '●', ha='center', va='center', fontsize=40, 
            transform=axes[2].transAxes, color='blue')
axes[2].text(0.5, 0.55, '●', ha='center', va='center', fontsize=40, 
            transform=axes[2].transAxes, color='blue')
axes[2].text(0.8, 0.55, '●', ha='center', va='center', fontsize=40, 
            transform=axes[2].transAxes, color='blue')
axes[2].text(0.5, 0.4, '⋮\n(784 total)', ha='center', va='center', 
            fontsize=20, transform=axes[2].transAxes)
axes[2].text(0.3, 0.25, '●', ha='center', va='center', fontsize=40, 
            transform=axes[2].transAxes, color='blue')
axes[2].text(0.7, 0.25, '●', ha='center', va='center', fontsize=40, 
            transform=axes[2].transAxes, color='blue')

axes[2].text(0.5, 0.05, 'Each neuron receives\nONE pixel value', 
            ha='center', va='bottom', fontsize=11, style='italic',
            transform=axes[2].transAxes)
axes[2].axis('off')

plt.tight_layout()
plt.show()

print("\n🎯 THE CONNECTION:")
print("  1. Start with 28×28 image (2D grid)")
print("  2. Flatten to 784-element vector (1D array)")
print("  3. Feed each element to ONE input neuron (784 neurons)")
print("\n💡 WHY 784 INPUT NEURONS?")
print("  Because we have 784 pixel values to process!")
print("  Each neuron is responsible for ONE pixel location.")

## Part 6: Data Preprocessing - Preparing for the Neural Network

Before training, we need to:
1. **Normalize** pixel values from [0, 255] to [0, 1]
2. **One-hot encode** labels (e.g., digit "3" → [0,0,0,1,0,0,0,0,0,0])
3. **Split** into training and test sets

In [ ]:
# Step 1: Normalize pixel values
print("🔧 STEP 1: Normalizing pixel values\n")
print(f"Before normalization: min={X.min()}, max={X.max()}")

X_normalized = X / 255.0

print(f"After normalization:  min={X_normalized.min()}, max={X_normalized.max()}")
print("\n💡 Why normalize?")
print("  • Neural networks work better with small numbers (0-1 instead of 0-255)")
print("  • Helps with gradient descent and prevents numerical instability")
print("  • Standard practice in machine learning!")

In [ ]:
# Step 2: One-hot encoding
def to_one_hot(y, num_classes=10):
    """Convert class labels to one-hot encoded vectors"""
    one_hot = np.zeros((len(y), num_classes))
    one_hot[np.arange(len(y)), y] = 1
    return one_hot

y_one_hot = to_one_hot(y)

print("🔧 STEP 2: One-hot encoding labels\n")
print("Examples of one-hot encoding:")
for i in range(5):
    print(f"  Digit {y[i]} → {y_one_hot[i]}")

print("\n💡 Why one-hot encoding?")
print("  • Neural network outputs 10 probabilities (one per digit)")
print("  • We need labels in the same format for comparison")
print("  • [0,0,0,1,0,0,0,0,0,0] means 'this is definitely a 3'")
print("  • Network learns to output similar patterns!")

In [ ]:
# Step 3: Split into training and test sets
# Standard MNIST split: first 60,000 for training, last 10,000 for testing
X_train = X_normalized[:60000]
y_train = y_one_hot[:60000]
y_train_labels = y[:60000]

X_test = X_normalized[60000:]
y_test = y_one_hot[60000:]
y_test_labels = y[60000:]

print("🔧 STEP 3: Splitting dataset\n")
print(f"Training set:")
print(f"  • Images: {X_train.shape[0]:,}")
print(f"  • Each image: {X_train.shape[1]} pixels")
print(f"  • Labels: {y_train.shape[0]:,} one-hot vectors (each is 10 elements)\n")

print(f"Test set:")
print(f"  • Images: {X_test.shape[0]:,}")
print(f"  • Each image: {X_test.shape[1]} pixels")
print(f"  • Labels: {y_test.shape[0]:,} one-hot vectors\n")

print("✅ Data ready for training!")
print("\n💡 Why separate train/test?")
print("  • Train on 60,000 images to learn patterns")
print("  • Test on 10,000 UNSEEN images to measure real performance")
print("  • Prevents overfitting - ensures the network generalizes!")

## Part 7: Network Architecture Explained in Detail

We'll build a 3-layer neural network:

```
Input Layer          Hidden Layer        Output Layer
784 neurons    →     128 neurons    →    10 neurons
[each pixel]         [features]          [digit 0-9]
                     ReLU                Softmax
```

### Why this architecture?

**Input Layer (784 neurons):**
- One neuron per pixel (28×28 = 784)
- Each receives one pixel's brightness value

**Hidden Layer (128 neurons):**
- Learns to detect features (edges, curves, loops)
- 128 is a good balance: enough to learn patterns, not too many to overtrain
- Uses ReLU activation: ReLU(x) = max(0, x)

**Output Layer (10 neurons):**
- One neuron per digit (0, 1, 2, ..., 9)
- Uses Softmax: converts scores to probabilities that sum to 1
- The neuron with highest probability is the prediction!

### How many parameters?
- Input → Hidden: (784 × 128) + 128 bias = **100,480**
- Hidden → Output: (128 × 10) + 10 bias = **1,290**
- **Total: 101,770 parameters** that the network will learn!

In [ ]:
# Define activation functions with detailed explanations

def relu(x):
    """ReLU: Rectified Linear Unit - outputs x if x>0, else 0"""
    return np.maximum(0, x)

def relu_derivative(x):
    """Derivative of ReLU: 1 if x>0, else 0"""
    return (x > 0).astype(float)

def softmax(x):
    """Softmax: converts scores to probabilities that sum to 1"""
    exp_x = np.exp(x - np.max(x, axis=1, keepdims=True))  # Stability trick
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

# Visualize activation functions
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# ReLU visualization
x = np.linspace(-3, 3, 200)
y_relu = relu(x)

axes[0].plot(x, y_relu, 'b-', linewidth=3, label='ReLU(x) = max(0, x)')
axes[0].axhline(y=0, color='k', linestyle='-', alpha=0.3)
axes[0].axvline(x=0, color='r', linestyle='--', alpha=0.5, linewidth=2)
axes[0].grid(True, alpha=0.3)
axes[0].set_xlabel('Input (x)', fontsize=13)
axes[0].set_ylabel('Output', fontsize=13)
axes[0].set_title('ReLU Activation Function', fontsize=15, fontweight='bold')
axes[0].legend(fontsize=11)
axes[0].text(-2.5, 2, 'Negative inputs → 0\nPositive inputs → pass through', 
            fontsize=11, bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

# Softmax visualization example
raw_scores = np.array([2.0, 1.0, 0.1, -1.0, -2.0])
probabilities = softmax(raw_scores.reshape(1, -1))[0]

x_pos = np.arange(len(raw_scores))
axes[1].bar(x_pos, probabilities, color=['red', 'orange', 'yellow', 'lightblue', 'lightblue'],
           edgecolor='black', linewidth=1.5)
axes[1].set_xlabel('Output Neuron', fontsize=13)
axes[1].set_ylabel('Probability', fontsize=13)
axes[1].set_title('Softmax Example: Scores → Probabilities', fontsize=15, fontweight='bold')
axes[1].set_xticks(x_pos)
axes[1].set_xticklabels([f'#{i}' for i in range(5)])
axes[1].grid(True, alpha=0.3, axis='y')

# Add value labels
for i, (score, prob) in enumerate(zip(raw_scores, probabilities)):
    axes[1].text(i, prob + 0.02, f'{prob:.3f}', ha='center', fontsize=10, fontweight='bold')
    axes[1].text(i, -0.05, f'score: {score}', ha='center', fontsize=9, style='italic')

axes[1].text(2, 0.4, f'Sum = {probabilities.sum():.1f}\n(always 1.0!)', 
            ha='center', fontsize=11, bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5))

plt.tight_layout()
plt.show()

print("\n✅ Activation functions explained:")
print("\n🔵 ReLU (for hidden layer):")
print("  • Simple: if input > 0, pass it through; else output 0")
print("  • Fast to compute")
print("  • Helps network learn non-linear patterns")
print("  • Example: ReLU(-2) = 0, ReLU(3) = 3")
print("\n🔴 Softmax (for output layer):")
print("  • Converts raw scores to probabilities")
print("  • All outputs sum to 1.0")
print("  • Largest score gets largest probability")
print("  • Network can express confidence: 'I'm 95% sure this is a 3'")

## Part 8: Building the Neural Network Class

Now we'll implement the complete neural network with detailed comments!

In [ ]:
class DigitRecognitionNetwork:
    def __init__(self, input_size=784, hidden_size=128, output_size=10):
        """
        Initialize the neural network with random weights
        
        Parameters:
        - input_size: 784 (one per pixel)
        - hidden_size: 128 (number of hidden neurons)
        - output_size: 10 (one per digit 0-9)
        """
        # Xavier initialization: smart random values that help training
        # Weights between input and hidden layer
        self.W1 = np.random.randn(input_size, hidden_size) * np.sqrt(2.0 / input_size)
        self.b1 = np.zeros((1, hidden_size))  # Biases for hidden layer
        
        # Weights between hidden and output layer
        self.W2 = np.random.randn(hidden_size, output_size) * np.sqrt(2.0 / hidden_size)
        self.b2 = np.zeros((1, output_size))  # Biases for output layer
        
        # Track training progress
        self.loss_history = []
        self.accuracy_history = []
        
    def forward(self, X):
        """
        Forward propagation: compute predictions
        
        X shape: (batch_size, 784) - batch of flattened images
        Returns: (batch_size, 10) - probabilities for each digit
        """
        # Layer 1: Input → Hidden
        # z1 = X · W1 + b1  (linear transformation)
        self.z1 = np.dot(X, self.W1) + self.b1  # Shape: (batch_size, 128)
        self.a1 = relu(self.z1)                  # Apply ReLU activation
        
        # Layer 2: Hidden → Output
        # z2 = a1 · W2 + b2  (linear transformation)
        self.z2 = np.dot(self.a1, self.W2) + self.b2  # Shape: (batch_size, 10)
        self.a2 = softmax(self.z2)                     # Apply Softmax
        
        return self.a2  # Probabilities for each digit
    
    def compute_loss(self, y_true, y_pred):
        """
        Compute cross-entropy loss
        
        Lower loss = better predictions
        """
        m = y_true.shape[0]  # Number of examples
        # Cross-entropy: -log(probability of correct class)
        log_likelihood = -np.log(y_pred[range(m), y_true.argmax(axis=1)] + 1e-8)
        loss = np.sum(log_likelihood) / m
        return loss
    
    def backward(self, X, y_true, learning_rate=0.01):
        """
        Backpropagation: compute gradients and update weights
        
        This is where the learning happens!
        """
        m = X.shape[0]
        
        # Step 1: Compute gradients for output layer
        dz2 = self.a2 - y_true  # Error at output
        dW2 = np.dot(self.a1.T, dz2) / m  # Gradient for W2
        db2 = np.sum(dz2, axis=0, keepdims=True) / m  # Gradient for b2
        
        # Step 2: Compute gradients for hidden layer
        dz1 = np.dot(dz2, self.W2.T) * relu_derivative(self.z1)  # Error at hidden
        dW1 = np.dot(X.T, dz1) / m  # Gradient for W1
        db1 = np.sum(dz1, axis=0, keepdims=True) / m  # Gradient for b1
        
        # Step 3: Update weights (gradient descent)
        self.W2 -= learning_rate * dW2
        self.b2 -= learning_rate * db2
        self.W1 -= learning_rate * dW1
        self.b1 -= learning_rate * db1
    
    def train(self, X_train, y_train, X_test, y_test, epochs=10, batch_size=128, learning_rate=0.1):
        """
        Train the network using mini-batch gradient descent
        
        Mini-batch: instead of using all 60,000 images at once,
        we use small batches of 128 images for efficiency
        """
        n_batches = len(X_train) // batch_size
        
        print(f"Training for {epochs} epochs with batch size {batch_size}")
        print(f"Number of batches per epoch: {n_batches}\n")
        
        for epoch in range(epochs):
            # Shuffle training data each epoch (prevents memorization)
            indices = np.random.permutation(len(X_train))
            X_shuffled = X_train[indices]
            y_shuffled = y_train[indices]
            
            # Train on mini-batches
            for i in range(n_batches):
                start_idx = i * batch_size
                end_idx = start_idx + batch_size
                
                X_batch = X_shuffled[start_idx:end_idx]
                y_batch = y_shuffled[start_idx:end_idx]
                
                # Forward pass: make predictions
                self.forward(X_batch)
                # Backward pass: learn from mistakes
                self.backward(X_batch, y_batch, learning_rate)
            
            # Evaluate performance after each epoch
            train_pred = self.forward(X_train)
            test_pred = self.forward(X_test)
            
            train_loss = self.compute_loss(y_train, train_pred)
            test_loss = self.compute_loss(y_test, test_pred)
            
            train_acc = np.mean(np.argmax(train_pred, axis=1) == np.argmax(y_train, axis=1))
            test_acc = np.mean(np.argmax(test_pred, axis=1) == np.argmax(y_test, axis=1))
            
            self.loss_history.append(test_loss)
            self.accuracy_history.append(test_acc)
            
            print(f"Epoch {epoch+1:2d}/{epochs} | "
                  f"Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | "
                  f"Test Loss: {test_loss:.4f} Acc: {test_acc:.4f}")
        
        print(f"\n✅ Training complete!")
        print(f"Final test accuracy: {test_acc:.4f} ({test_acc*100:.2f}%)")
    
    def predict(self, X):
        """Make predictions: return the digit with highest probability"""
        probabilities = self.forward(X)
        return np.argmax(probabilities, axis=1)

# Create the network
print("🧠 Creating Neural Network...\n")
nn = DigitRecognitionNetwork(input_size=784, hidden_size=128, output_size=10)

print("Architecture Summary:")
print(f"  Input Layer:    784 neurons (one per pixel)")
print(f"  Hidden Layer:   128 neurons (ReLU activation)")
print(f"  Output Layer:    10 neurons (Softmax activation)")
print(f"\nParameters:")
print(f"  W1 shape: {nn.W1.shape} → {nn.W1.size:,} weights")
print(f"  b1 shape: {nn.b1.shape} → {nn.b1.size:,} biases")
print(f"  W2 shape: {nn.W2.shape} → {nn.W2.size:,} weights")
print(f"  b2 shape: {nn.b2.shape} → {nn.b2.size:,} biases")
print(f"  Total parameters: {nn.W1.size + nn.b1.size + nn.W2.size + nn.b2.size:,}")
print("\n✅ Network ready to train!")

## Part 9: Train the Network! 🚀

Now for the exciting part - watching the network learn!

**What to watch for:**
- Loss should decrease (network getting better at predicting)
- Accuracy should increase (more correct predictions)
- Should reach >95% accuracy after 10 epochs

In [ ]:
# Train the network
print("🏋️ Training on 60,000 handwritten digits...\n")
print("This will take 2-3 minutes. Watch the accuracy improve!\n")

nn.train(
    X_train, y_train,
    X_test, y_test,
    epochs=10,
    batch_size=128,
    learning_rate=0.1
)

In [ ]:
# Visualize training progress
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Loss over time
epochs_range = range(1, len(nn.loss_history) + 1)
axes[0].plot(epochs_range, nn.loss_history, 'b-o', linewidth=2, markersize=8)
axes[0].set_xlabel('Epoch', fontsize=13)
axes[0].set_ylabel('Loss (Cross-Entropy)', fontsize=13)
axes[0].set_title('Training Loss Over Time\n(Lower is Better)', fontsize=15, fontweight='bold')
axes[0].grid(True, alpha=0.3)
axes[0].set_xticks(epochs_range)

# Accuracy over time
accuracy_pct = [acc * 100 for acc in nn.accuracy_history]
axes[1].plot(epochs_range, accuracy_pct, 'g-o', linewidth=2, markersize=8)
axes[1].set_xlabel('Epoch', fontsize=13)
axes[1].set_ylabel('Test Accuracy (%)', fontsize=13)
axes[1].set_title('Test Accuracy Over Time\n(Higher is Better)', fontsize=15, fontweight='bold')
axes[1].grid(True, alpha=0.3)
axes[1].set_ylim([0, 100])
axes[1].set_xticks(epochs_range)
axes[1].axhline(y=95, color='r', linestyle='--', alpha=0.5, label='95% target')
axes[1].legend()

plt.tight_layout()
plt.show()

print("\n📈 Performance Summary:")
print(f"  • Starting accuracy: {nn.accuracy_history[0]*100:.2f}%")
print(f"  • Final accuracy: {nn.accuracy_history[-1]*100:.2f}%")
print(f"  • Improvement: +{(nn.accuracy_history[-1] - nn.accuracy_history[0])*100:.2f} percentage points")
print(f"  • Starting loss: {nn.loss_history[0]:.4f}")
print(f"  • Final loss: {nn.loss_history[-1]:.4f}")

if nn.accuracy_history[-1] > 0.95:
    print("\n🏆 Excellent! Achieved >95% accuracy!")
elif nn.accuracy_history[-1] > 0.90:
    print("\n👍 Good performance! Above 90%.")
else:
    print("\n💪 Network is learning. Try more epochs for better results.")

## Part 10: Test the Trained Network

Let's see how well it performs on images it has NEVER seen before!

In [ ]:
# Make predictions on test set
print("🎯 Evaluating on Test Set (10,000 unseen images)...\n")

predictions = nn.predict(X_test)
accuracy = accuracy_score(y_test_labels, predictions)

print(f"Results:")
print(f"  • Total test images: {len(X_test):,}")
print(f"  • Correct predictions: {int(accuracy * len(X_test)):,}")
print(f"  • Incorrect predictions: {len(X_test) - int(accuracy * len(X_test)):,}")
print(f"  • Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")

if accuracy > 0.95:
    print("\n🏆 Outstanding! Professional-grade performance!")
    print("    This is better than many commercial systems from the 1990s!")
elif accuracy > 0.90:
    print("\n👍 Strong performance! The network learned well.")
else:
    print("\n⚡ Room for improvement. Consider more training epochs or tuning hyperparameters.")

In [ ]:
# Visualize sample predictions with probabilities
fig, axes = plt.subplots(4, 5, figsize=(16, 13))
fig.suptitle('Sample Predictions from Test Set', fontsize=18, fontweight='bold')

for i, ax in enumerate(axes.flat):
    idx = np.random.randint(0, len(X_test))
    image = X_test[idx].reshape(28, 28)
    true_label = y_test_labels[idx]
    pred_label = predictions[idx]
    
    # Get probabilities
    probs = nn.forward(X_test[idx:idx+1])[0]
    confidence = probs[pred_label]
    
    ax.imshow(image, cmap='gray')
    
    if true_label == pred_label:
        ax.set_title(f'✅ Predicted: {pred_label} (confidence: {confidence:.2%})\nTrue: {true_label}', 
                    fontsize=10, color='green', fontweight='bold')
    else:
        ax.set_title(f'❌ Predicted: {pred_label} (confidence: {confidence:.2%})\nTrue: {true_label}', 
                    fontsize=10, color='red', fontweight='bold')
    
    ax.axis('off')

plt.tight_layout()
plt.show()

print("\n💡 Understanding the predictions:")
print("  • Green = correct prediction")
print("  • Red = incorrect prediction (rare!)")
print("  • Confidence = how sure the network is (0-100%)")
print("  • High confidence + correct = network learned well!")

## Part 11: Confusion Matrix - Understanding Mistakes

A confusion matrix shows which digits the network confuses with each other.

**How to read it:**
- Rows = true labels (what the digit actually is)
- Columns = predicted labels (what the network guessed)
- Diagonal = correct predictions (should be largest numbers)
- Off-diagonal = mistakes (smaller numbers)

In [ ]:
# Compute and visualize confusion matrix
cm = confusion_matrix(y_test_labels, predictions)

plt.figure(figsize=(12, 10))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
           xticklabels=range(10), yticklabels=range(10),
           cbar_kws={'label': 'Number of Images'})
plt.xlabel('Predicted Digit', fontsize=14, fontweight='bold')
plt.ylabel('True Digit', fontsize=14, fontweight='bold')
plt.title('Confusion Matrix\n(Bright diagonal = Good Performance)', 
          fontsize=16, fontweight='bold', pad=20)
plt.tight_layout()
plt.show()

print("\n📊 How to interpret this:")
print("  • Each cell shows: 'how many times true digit X was predicted as Y'")
print("  • Diagonal (top-left to bottom-right) = correct predictions")
print("  • Bright diagonal with dark off-diagonal = excellent performance!")
print("\n🔍 Common confusions (if any):")

# Find most common mistakes
mistakes = []
for i in range(10):
    for j in range(10):
        if i != j and cm[i, j] > 5:  # More than 5 mistakes
            mistakes.append((i, j, cm[i, j]))

mistakes.sort(key=lambda x: x[2], reverse=True)

if len(mistakes) > 0:
    for true_digit, pred_digit, count in mistakes[:5]:
        print(f"  • Digit '{true_digit}' confused as '{pred_digit}': {count} times")
    print("\n💡 These confusions make sense! Try writing a 4 and 9 - they can look similar!")
else:
    print("  • Very few mistakes! Excellent performance across all digits!")

## Part 12: Visualize What the Network Learned

Let's peek inside the hidden layer to see what features it learned to detect!

**What you'll see:**
- Each hidden neuron has 784 weights (one per pixel)
- We can reshape these weights back into 28×28 to visualize
- Red areas = neuron looks for bright pixels there
- Blue areas = neuron looks for dark pixels there
- These are the FEATURES the network discovered on its own!

In [ ]:
# Visualize hidden layer weights as learned features
fig, axes = plt.subplots(8, 16, figsize=(20, 10))
fig.suptitle('All 128 Hidden Layer Features (What Each Neuron "Looks For")', 
             fontsize=18, fontweight='bold')

for i, ax in enumerate(axes.flat):
    # Get the weights for this hidden neuron and reshape to 28×28
    weights = nn.W1[:, i].reshape(28, 28)
    
    ax.imshow(weights, cmap='RdBu', vmin=-1, vmax=1)
    ax.set_title(f'#{i}', fontsize=8)
    ax.axis('off')

plt.tight_layout()
plt.show()

print("\n🧠 What are we seeing?")
print("  • Each image = what ONE hidden neuron learned to detect")
print("  • Red regions = neuron activates when pixels are BRIGHT there")
print("  • Blue regions = neuron activates when pixels are DARK there")
print("  • White = neuron doesn't care about those pixels")
print("\n💡 Amazing fact: We NEVER told the network what to look for!")
print("   It discovered these features on its own through training!")
print("\n🔍 Look for patterns like:")
print("  • Edge detectors (vertical/horizontal lines)")
print("  • Curve detectors (for recognizing rounded digits like 0, 6, 8, 9)")
print("  • Corner detectors")
print("  • Specific digit patterns")

In [ ]:
# Show a specific example: how hidden neurons respond to a specific digit
test_idx = 42  # Pick any test image
test_image = X_test[test_idx]
test_label = y_test_labels[test_idx]

# Forward pass to get hidden layer activations
_ = nn.forward(test_image.reshape(1, -1))
hidden_activations = nn.a1[0]  # Activations of all 128 hidden neurons

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

# Show the input image
axes[0].imshow(test_image.reshape(28, 28), cmap='gray')
axes[0].set_title(f'Input Image: Digit {test_label}', fontsize=14, fontweight='bold')
axes[0].axis('off')

# Show hidden layer activations
axes[1].bar(range(len(hidden_activations)), hidden_activations, color='steelblue')
axes[1].set_xlabel('Hidden Neuron ID (0-127)', fontsize=12)
axes[1].set_ylabel('Activation Level', fontsize=12)
axes[1].set_title('Hidden Layer Response\n(How much each neuron activated)', 
                  fontsize=14, fontweight='bold')
axes[1].grid(True, alpha=0.3, axis='y')

# Show output layer probabilities
output_probs = nn.a2[0]
colors = ['green' if i == test_label else 'lightblue' for i in range(10)]
axes[2].bar(range(10), output_probs, color=colors, edgecolor='black', linewidth=2)
axes[2].set_xlabel('Digit', fontsize=12)
axes[2].set_ylabel('Probability', fontsize=12)
axes[2].set_title(f'Output Probabilities\n(Prediction: {np.argmax(output_probs)})', 
                  fontsize=14, fontweight='bold')
axes[2].set_xticks(range(10))
axes[2].grid(True, alpha=0.3, axis='y')

for i, prob in enumerate(output_probs):
    if prob > 0.01:  # Only show significant probabilities
        axes[2].text(i, prob + 0.02, f'{prob:.3f}', ha='center', fontsize=9)

plt.tight_layout()
plt.show()

print(f"\n🔍 Flow of information for this digit {test_label}:")
print(f"  1. Input: 784 pixel values (the flattened image)")
print(f"  2. Hidden: 128 feature detectors activate (middle graph shows which ones)")
print(f"  3. Output: 10 probabilities computed (right graph)")
print(f"  4. Prediction: Digit with highest probability = {np.argmax(output_probs)}")
print(f"\n💡 Notice:")
print(f"  • Different hidden neurons activate for different digits")
print(f"  • The network is {output_probs[test_label]*100:.1f}% confident this is a {test_label}")

## Part 13: Interactive Prediction - Try Your Own!

Let's make predictions on random test images and see the network's reasoning!

In [ ]:
# Interactive prediction function
def show_prediction_detail(index):
    """
    Show detailed prediction for a specific test image
    """
    image = X_test[index].reshape(28, 28)
    true_label = y_test_labels[index]
    
    # Get prediction
    probs = nn.forward(X_test[index:index+1])[0]
    predicted_label = np.argmax(probs)
    confidence = probs[predicted_label]
    
    # Create visualization
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Show image
    axes[0].imshow(image, cmap='gray')
    axes[0].set_title(f'Test Image #{index}\nTrue Label: {true_label}', 
                      fontsize=14, fontweight='bold')
    axes[0].axis('off')
    
    # Show probabilities
    colors = ['green' if i == predicted_label else 'lightblue' for i in range(10)]
    axes[1].bar(range(10), probs, color=colors, edgecolor='black', linewidth=2)
    axes[1].set_xlabel('Digit', fontsize=13)
    axes[1].set_ylabel('Probability', fontsize=13)
    axes[1].set_title(f'Network Prediction: {predicted_label} (confidence: {confidence:.1%})', 
                      fontsize=14, fontweight='bold')
    axes[1].set_xticks(range(10))
    axes[1].grid(True, alpha=0.3, axis='y')
    
    # Add probability labels
    for i, prob in enumerate(probs):
        if prob > 0.01:
            axes[1].text(i, prob + 0.01, f'{prob:.3f}', ha='center', fontsize=9)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary
    if predicted_label == true_label:
        print(f"✅ CORRECT! The network correctly identified this as a {true_label}")
        print(f"   Confidence: {confidence:.2%}")
    else:
        print(f"❌ INCORRECT! The network predicted {predicted_label} but it's actually {true_label}")
        print(f"   Confidence in wrong answer: {confidence:.2%}")
        print(f"   Confidence in correct answer: {probs[true_label]:.2%}")
    
    print(f"\nTop 3 predictions:")
    top3_indices = np.argsort(probs)[-3:][::-1]
    for rank, idx in enumerate(top3_indices, 1):
        print(f"  {rank}. Digit {idx}: {probs[idx]:.2%}")

# Try a few random examples
print("🎲 Let's examine some random predictions:\n")
for _ in range(3):
    random_idx = np.random.randint(0, len(X_test))
    show_prediction_detail(random_idx)
    print("\n" + "="*80 + "\n")

## 🎓 Summary: What You've Learned

### Core Concepts Mastered:

**1. Image Flattening (THE KEY CONCEPT!)**
- 2D images (28×28) → 1D vectors (784 elements)
- Row-by-row concatenation preserves all information
- Each pixel value maps to exactly one input neuron

**2. Neural Network Architecture**
- Input layer: 784 neurons (one per pixel)
- Hidden layer: 128 neurons with ReLU activation (feature detectors)
- Output layer: 10 neurons with Softmax (probability distribution)
- Total: 101,770 trainable parameters

**3. The Training Process**
- Forward propagation: input → hidden → output
- Loss computation: measuring prediction error
- Backpropagation: computing gradients
- Gradient descent: updating weights to reduce error

**4. Evaluation**
- Training set: learn patterns (60,000 images)
- Test set: measure generalization (10,000 unseen images)
- Achieved >95% accuracy on handwritten digits!

### Why This Matters for Your Assignment:

**Text Classification uses the SAME principles:**
- Text → Vector (instead of Image → Vector)
- Same neural network architecture
- Same forward/backward propagation
- Same training loop

**The simplified assignment gives you:**
- Pre-processed text (we handle the "flattening" for you)
- 500 movie reviews (manageable dataset size)
- 3 focused TODOs (forward pass, loss, training)
- 2-4 hours (achievable timeframe)

### Next Steps:

1. **Review this notebook** - make sure you understand flattening!
2. **Try the simplified assignment** - apply what you learned
3. **Experiment** - change hyperparameters, observe effects
4. **Ask questions** - if anything is unclear, ask now!

### Key Takeaway:

**Neural networks are pattern-matching machines:**
- We give them data (images, text, anything!)
- We show them examples (training)
- They learn to recognize patterns (weights)
- They can generalize to new data (testing)

**You now understand how they work! 🎉**

## 🔬 Bonus: Experiment Ideas

Want to explore further? Try these experiments:

1. **Change the hidden layer size:**
   - Try 64 neurons: Faster but less accurate?
   - Try 256 neurons: More accurate but slower?

2. **Adjust learning rate:**
   - Try 0.01: Slower learning?
   - Try 0.5: Faster but unstable?

3. **Train for more epochs:**
   - Does accuracy keep improving?
   - When does it plateau?

4. **Analyze mistakes:**
   - Find all misclassified images
   - Are they hard for humans too?
   - What patterns do you notice?

5. **Add a second hidden layer:**
   - Does deeper = better?
   - What's the performance/speed tradeoff?

**Happy experimenting! 🚀**